/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 


/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [1]:
import sqlite3
import pandas as pd

In [2]:
with sqlite3.connect("sqlite_db_pythonsqlite.db") as conn:
    curr = conn.cursor()
    data = curr.execute("SELECT * FROM Facilities")
    df = pd.DataFrame(data.fetchall())
    names = curr.description
columns = list(map(lambda x: x[0], names))
df.columns = columns
df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [3]:
with sqlite3.connect("sqlite_db_pythonsqlite.db") as conn:
    curr = conn.cursor()
    data = curr.execute("""SELECT  
    facility,
    SUM(cost) AS revenue
FROM (
    SELECT name AS facility, facid,
    CASE WHEN memid = 0 THEN slots * guestcost
        ELSE slots * membercost END AS cost
    FROM Bookings
    INNER JOIN Facilities USING(facid)
    INNER JOIN Members USING(memid)
    ) AS subquery
GROUP BY facid
HAVING revenue < 1000
ORDER BY revenue DESC;""")
    df = pd.DataFrame(data.fetchall())
    names = curr.description
columns = list(map(lambda x: x[0], names))
df.columns = columns
df



,facility,revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [4]:
with sqlite3.connect("sqlite_db_pythonsqlite.db") as conn:
    curr = conn.cursor()
    data = curr.execute("""SELECT  
     m1.surname || ", " || m1.firstname AS membername, 
     m2.surname || ", " || m2.firstname AS referrer
FROM Members AS m1
LEFT JOIN Members AS m2
ON m1.recommendedby = m2.memid
WHERE m1.memid <> 0
ORDER BY membername;""")
    df = pd.DataFrame(data.fetchall())
    names = curr.description
columns = list(map(lambda x: x[0], names))
df.columns = columns
df.style.hide_index()


membername,referrer
"Bader, Florence","Stibbons, Ponder"
"Baker, Anne","Stibbons, Ponder"
"Baker, Timothy","Farrell, Jemima"
"Boothe, Tim","Rownam, Tim"
"Butters, Gerald","Smith, Darren"
"Coplin, Joan","Baker, Timothy"
"Crumpet, Erica","Smith, Tracy"
"Dare, Nancy","Joplette, Janice"
"Farrell, David",None
"Farrell, Jemima",None


/* Q12: Find the facilities with their usage by member, but not guests */

In [5]:
with sqlite3.connect("sqlite_db_pythonsqlite.db") as conn:
    curr = conn.cursor()
    data = curr.execute("""SELECT  
        name AS facility,
        COUNT(DISTINCT memid) AS member_usage
    FROM Bookings
    LEFT JOIN Facilities USING(facid)
    WHERE memid <> 0
    GROUP BY facid
    ORDER BY facid;""")
    df = pd.DataFrame(data.fetchall())
    names = curr.description
columns = list(map(lambda x: x[0], names))
df.columns = columns
print(df.to_string(index=False))

       facility  member_usage
 Tennis Court 1            23
 Tennis Court 2            21
Badminton Court            24
   Table Tennis            25
 Massage Room 1            24
 Massage Room 2            12
   Squash Court            24
  Snooker Table            22
     Pool Table            27


In [6]:
with sqlite3.connect("sqlite_db_pythonsqlite.db") as conn:
    curr = conn.cursor()
    data = curr.execute("""SELECT  
        name AS facility,
        SUM(slots*0.5) AS usage_hours
    FROM Bookings
    Left JOIN Facilities USING(facid)
    WHERE memid <> 0
    GROUP BY facid
    ORDER BY facid;""")
    df = pd.DataFrame(data.fetchall())
    names = curr.description
columns = list(map(lambda x: x[0], names))
df.columns = columns
print(df.to_string(index=False))

       facility  usage_hours
 Tennis Court 1        478.5
 Tennis Court 2        441.0
Badminton Court        543.0
   Table Tennis        397.0
 Massage Room 1        442.0
 Massage Room 2         27.0
   Squash Court        209.0
  Snooker Table        430.0
     Pool Table        428.0


In [7]:
with sqlite3.connect("sqlite_db_pythonsqlite.db") as conn:
    curr = conn.cursor()
    data = curr.execute("""SELECT  
        name AS facility, surname || ", " || firstname AS member,
        COUNT(memid) AS member_usage
    FROM Bookings
    LEFT JOIN Facilities USING(facid)
    LEFT JOIN Members USING (memid)
    WHERE memid <> 0
    GROUP BY facid, memid
    ORDER BY facid, memid;""")
    df = pd.DataFrame(data.fetchall())
    names = curr.description
columns = list(map(lambda x: x[0], names))
df.columns = columns
df.set_index(['facility', 'member'], inplace=True)
print(df.to_string())

                                          member_usage
facility        member                                
Tennis Court 1  Smith, Tracy                        30
                Rownam, Tim                          6
                Joplette, Janice                    19
                Butters, Gerald                     57
                Tracy, Burton                       31
                Dare, Nancy                         25
                Boothe, Tim                          4
                Stibbons, Ponder                     1
                Owen, Charles                       17
                Jones, David                        25
                Baker, Anne                          6
                Farrell, Jemima                      1
                Smith, Jack                         22
                Bader, Florence                      1
                Baker, Timothy                      14
                Pinker, David                       16
          

In [8]:
with sqlite3.connect("sqlite_db_pythonsqlite.db") as conn:
    curr = conn.cursor()
    data = curr.execute("""SELECT  
        name AS facility, surname || ", " || firstname AS member,
        SUM(slots*0.5) AS usage_hours
    FROM Bookings
    LEFT JOIN Facilities USING(facid)
    LEFT JOIN Members USING(memid)
    WHERE memid <> 0
    GROUP BY facid, memid
    ORDER BY facid, memid;""")
    df = pd.DataFrame(data.fetchall())
    names = curr.description
columns = list(map(lambda x: x[0], names))
df.columns = columns
df.set_index(['facility', 'member'], inplace=True)
print(df.to_string())

                                          usage_hours
facility        member                               
Tennis Court 1  Smith, Tracy                     46.5
                Rownam, Tim                       9.0
                Joplette, Janice                 28.5
                Butters, Gerald                  85.5
                Tracy, Burton                    46.5
                Dare, Nancy                      40.5
                Boothe, Tim                       6.0
                Stibbons, Ponder                  1.5
                Owen, Charles                    25.5
                Jones, David                     42.0
                Baker, Anne                       9.0
                Farrell, Jemima                   1.5
                Smith, Jack                      37.5
                Bader, Florence                   1.5
                Baker, Timothy                   22.5
                Pinker, David                    25.5
                Genting, Mat

/* Q13: Find the facilities usage by month, but not guests */

In [9]:
with sqlite3.connect("sqlite_db_pythonsqlite.db") as conn:
    curr = conn.cursor()
    data = curr.execute("""SELECT  
        name AS facility, STRFTIME('%m', starttime) AS month,
        SUM(slots*0.5) AS usage_hours
    FROM Bookings
    LEFT JOIN Facilities USING(facid)
    WHERE memid <> 0
    GROUP BY facid, month
    ORDER BY facid, month;""")
    df = pd.DataFrame(data.fetchall())
    names = curr.description
columns = list(map(lambda x: x[0], names))
df.columns = columns
df.set_index(['facility', 'month'], inplace=True)
print(df.to_string())

                       usage_hours
facility        month             
Tennis Court 1  07           100.5
                08           169.5
                09           208.5
Tennis Court 2  07            61.5
                08           172.5
                09           207.0
Badminton Court 07            82.5
                08           207.0
                09           253.5
Table Tennis    07            49.0
                08           148.0
                09           200.0
Massage Room 1  07            83.0
                08           158.0
                09           201.0
Massage Room 2  07             4.0
                08             9.0
                09            14.0
Squash Court    07            25.0
                08            92.0
                09            92.0
Snooker Table   07            70.0
                08           158.0
                09           202.0
Pool Table      07            55.0
                08           151.5
                09  

In [10]:
with sqlite3.connect("sqlite_db_pythonsqlite.db") as conn:
    curr = conn.cursor()
    data = curr.execute("""SELECT  
        name AS facility, STRFTIME('%m', starttime) AS month,
        COUNT(memid) AS usage
    FROM Bookings
    LEFT JOIN Facilities USING(facid)
    WHERE memid <> 0
    GROUP BY facid, month
    ORDER BY facid, month;""")
    df = pd.DataFrame(data.fetchall())
    names = curr.description
columns = list(map(lambda x: x[0], names))
df.columns = columns
df.set_index(['facility', 'month'], inplace=True)
print(df.to_string())

                       usage
facility        month       
Tennis Court 1  07        65
                08       111
                09       132
Tennis Court 2  07        41
                08       109
                09       126
Badminton Court 07        51
                08       132
                09       161
Table Tennis    07        48
                08       143
                09       194
Massage Room 1  07        77
                08       153
                09       191
Massage Room 2  07         4
                08         9
                09        14
Squash Court    07        23
                08        85
                09        87
Snooker Table   07        68
                08       154
                09       199
Pool Table      07       103
                08       272
                09       408
